<a href="https://colab.research.google.com/github/sreent/relational-databases/blob/main/normalising_wide_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
columns = ["Category",	"Item",	"Serving Size (g)",	"Calories (kcal)",	"Calories from Fat (kcal)",	"Saturated Fat (g)",	
           "Cholesterol (mg)",	"Sodium (mg)",	"Carbohydrates (g)",	"Dietary Fiber (g)",	"Sugars (g)",	"Protein (g)",	
           "Vitamin A (% Daily Value)", "Vitamin C (% Daily Value)",	"Calcium (% Daily Value)",	"Iron (% Daily Value)"]

entries = [["Breakfast",	"Egg McMuffin",	136,	300,	120,	5,	260,	750,	31,	4,	3,	17,	10,	0,	25,	15],
           ["Beef & Pork",	"Big Mac",	211,	530,	240,	10,	85,	960,	47,	3,	9,	24,	6,	2,	25,	25]]


In [3]:
raw_data = pd.DataFrame(data=entries, columns=columns)
raw_data.head()

,Category,Item,Serving Size (g),Calories (kcal),Calories from Fat (kcal),Saturated Fat (g),Cholesterol (mg),Sodium (mg),Carbohydrates (g),Dietary Fiber (g),Sugars (g),Protein (g),Vitamin A (% Daily Value),Vitamin C (% Daily Value),Calcium (% Daily Value),Iron (% Daily Value)
0,Breakfast,Egg McMuffin,136,300,120,5,260,750,31,4,3,17,10,0,25,15
1,Beef & Pork,Big Mac,211,530,240,10,85,960,47,3,9,24,6,2,25,25


In [4]:
nutrients = list(raw_data.columns)[2:]
nutrients = dict(zip(range(len(nutrients)), nutrients))
nutrients

{0: 'Serving Size (g)',
 1: 'Calories (kcal)',
 2: 'Calories from Fat (kcal)',
 3: 'Saturated Fat (g)',
 4: 'Cholesterol (mg)',
 5: 'Sodium (mg)',
 6: 'Carbohydrates (g)',
 7: 'Dietary Fiber (g)',
 8: 'Sugars (g)',
 9: 'Protein (g)',
 10: 'Vitamin A (% Daily Value)',
 11: 'Vitamin C (% Daily Value)',
 12: 'Calcium (% Daily Value)',
 13: 'Iron (% Daily Value)'}

In [5]:
stub, sep = 'Quantity', '_'
raw_data.columns = list(raw_data.columns)[:2] + [stub + sep + str(k) for k in nutrients.keys()]
raw_data.head()

,Category,Item,Quantity_0,Quantity_1,Quantity_2,Quantity_3,Quantity_4,Quantity_5,Quantity_6,Quantity_7,Quantity_8,Quantity_9,Quantity_10,Quantity_11,Quantity_12,Quantity_13
0,Breakfast,Egg McMuffin,136,300,120,5,260,750,31,4,3,17,10,0,25,15
1,Beef & Pork,Big Mac,211,530,240,10,85,960,47,3,9,24,6,2,25,25


In [6]:
df = pd.wide_to_long(raw_data, stubnames="Quantity", i=["Category", "Item"],  j="Nutrient", sep="_", suffix=r"\w+")
df.reset_index(inplace=True)

df['Nutrient'] = df['Nutrient'].apply(lambda x: nutrients[x])
df['Unit'] = df['Nutrient'].apply(lambda x: x.split('(')[-1][:-1])
df['Nutrient'] = df['Nutrient'].apply(lambda x: x.split(' (')[0])

df.head()

,Category,Item,Nutrient,Quantity,Unit
0,Breakfast,Egg McMuffin,Serving Size,136,g
1,Breakfast,Egg McMuffin,Calories,300,kcal
2,Breakfast,Egg McMuffin,Calories from Fat,120,kcal
3,Breakfast,Egg McMuffin,Saturated Fat,5,g
4,Breakfast,Egg McMuffin,Cholesterol,260,mg


In [7]:
menus = pd.DataFrame(data=df['Category'].unique(), columns=['category'])
menus.reset_index(inplace=True)
menus.columns = ['id', 'menu']

menus.head()

,id,menu
0,0,Breakfast
1,1,Beef & Pork


In [8]:
meals = df[['Item', 'Category']].drop_duplicates()
meals.reset_index(drop=True, inplace=True)
meals.reset_index(inplace=True)
meals.columns = ['id', 'meal', 'menu']

mapping = dict(menus[['menu', 'id']].values)
meals['menu'] = meals['menu'].apply(lambda x: mapping[x])

meals.head()

,id,meal,menu
0,0,Egg McMuffin,0
1,1,Big Mac,1


In [9]:
nutrients = df[['Nutrient', 'Unit']].drop_duplicates()
nutrients.reset_index(drop=True, inplace=True)
nutrients.reset_index(inplace=True)
nutrients.columns = ['id', 'nutrient', 'unit']

nutrients.head()

,id,nutrient,unit
0,0,Serving Size,g
1,1,Calories,kcal
2,2,Calories from Fat,kcal
3,3,Saturated Fat,g
4,4,Cholesterol,mg


In [10]:
nutrition = df[['Item', 'Nutrient', 'Quantity']]
nutrition.columns = ['meal', 'nutrient', 'quantity']

mapping = dict(meals[['meal', 'id']].values)
nutrition['meal'] = nutrition['meal'].apply(lambda x: mapping[x])

mapping = dict(nutrients[['nutrient', 'id']].values)
nutrition['nutrient'] = nutrition['nutrient'].apply(lambda x: mapping[x])

nutrition.head(25)

<ipython-input-10-656f17dc4048>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nutrition['meal'] = nutrition['meal'].apply(lambda x: mapping[x])
<ipython-input-10-656f17dc4048>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nutrition['nutrient'] = nutrition['nutrient'].apply(lambda x: mapping[x])


,meal,nutrient,quantity
0,0,0,136
1,0,1,300
2,0,2,120
3,0,3,5
4,0,4,260
5,0,5,750
6,0,6,31
7,0,7,4
8,0,8,3
9,0,9,17
